# Readability assessment

## Description

We compute accuracy for BERT + 1-uplet of hand-craft features on Vikiwiki Dataset using 10-folds cross validation

In [4]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00


In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.metrics import confusion_matrix, classification_report,precision_score,accuracy_score,f1_score
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import spacy
import os
import sys

import logging
logging.basicConfig(level = logging.ERROR)


from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-1-dc20d75813f1>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def load_vikiwiki_dataset(path: str, label: str|int, difficulty: str, lang: str = "en") -> pd.DataFrame:
    cols = ["content", "labels", "difficulty"]
    result = []
    load_path = path
    if os.path.isdir(path):
        dir_file_list = os.listdir(path)
        dir_file_list.sort()
        if lang in dir_file_list:
            load_path = os.path.join(load_path, lang)
            dir_file_list = os.listdir(load_path)
            dir_file_list.sort()
        for file in dir_file_list:
            with open(os.path.join(load_path, file), "r") as f:
                data = "".join(f.readlines())
            if len(data) > 0:
                result.append([data, label, difficulty])
    return pd.DataFrame(
        data=result,
        columns=cols
    )

In [6]:
df_ca = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "ca"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "ca")
    ],
    axis = 0,
    ignore_index = True
)

df_en = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "en"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "en")
    ],
    axis = 0,
    ignore_index = True
)

df_es = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "es"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "es")
    ],
    axis = 0,
    ignore_index = True
)

df_eu = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "eu"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "eu")
    ],
    axis = 0,
    ignore_index = True
)

df_fr = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "fr"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "fr")
    ],
    axis = 0,
    ignore_index = True
)

df_it = pd.concat(
    [
        load_vikiwiki_dataset("/content/drive/MyDrive/data/vikidia", 0, "beginner", "it"),
        load_vikiwiki_dataset("/content/drive/MyDrive/data/wikipedia", 1, "advanced", "it")
    ],
    axis = 0,
    ignore_index = True
)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [8]:
num_diff = df_en.labels.unique()
diff = ['beginner', 'advanced']

label = dict(zip(num_diff, diff))

diff = list(diff)


Data Preprocessing

In [ ]:
import re

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:

def clean_text(text):
    #text=' '.join([contraction_mapping[i] if i in contraction_mapping.keys() else i for i in text.split()])
    text=re.sub("'s","",text)
    if text.split()[0] == 'Intermediate':
      text = ' '.join(text.split()[1:])
    text=' '.join([i for i in text.split() if i.isalpha()])
    text=re.sub('[^a-zA-Z]'," ",text)


    return text

In [ ]:
df_en['content'] = df_en['content'].apply(clean_text)

sentence lenth

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install lingfeat
from lingfeat import extractor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 35.2 MB/s eta 0:00:00
  Created wheel for lingfeat: filename=lingfeat-1.0.0b19-py3-none-any.whl size=71127048 sha256=c37f5cc6856436e62c801815db9976a7255693b6a9b54ea3f8e0bb9e978f6864
  Stored in directory: /root/.cache/pip/wheels/f0/6c/25/7cc4b4dbfc7c605bad9f8824afef1b4abc8ed18232972c86a8
Successfully built lingfeat


Downloading: https://github.com/yzhangcs/parser/releases/download/v1.1.0/ptb.crf.con.lstm.char.zip to /root/.cache/supar/ptb.crf.con.lstm.char.zip
100%|██████████| 334M/334M [00:05<00:00, 66.1MB/s]


In [ ]:
import gc

def get_avg_nounphrase(text):
  LingFeat = extractor.pass_text(text)
  LingFeat.preprocess()
  avg = LingFeat.PhrF_()['as_NoPhr_C']
  gc.collect()
  return avg

In [ ]:
df['content'][459:460].progress_apply(get_avg_nounphrase)


  0%|          | 0/1 [00:00<?, ?it/s]

{'to_NoPhr_C': 38, 'as_NoPhr_C': 19.0, 'at_NoPhr_C': 0.40860215053763443, 'ra_NoVeP_C': 2.533333333333333, 'ra_NoSuP_C': 38.0, 'ra_NoPrP_C': 2.0, 'ra_NoAjP_C': 0.0, 'ra_NoAvP_C': 12.666666666666666, 'to_VePhr_C': 15, 'as_VePhr_C': 7.5, 'at_VePhr_C': 0.16129032258064516, 'ra_VeNoP_C': 0.39473684210526316, 'ra_VeSuP_C': 15.0, 'ra_VePrP_C': 0.7894736842105263, 'ra_VeAjP_C': 0.0, 'ra_VeAvP_C': 5.0, 'to_SuPhr_C': 1, 'as_SuPhr_C': 0.5, 'at_SuPhr_C': 0.010752688172043012, 'ra_SuNoP_C': 0.02631578947368421, 'ra_SuVeP_C': 0.06666666666666667, 'ra_SuPrP_C': 0.05263157894736842, 'ra_SuAjP_C': 0.0, 'ra_SuAvP_C': 0.3333333333333333, 'to_PrPhr_C': 19, 'as_PrPhr_C': 9.5, 'at_PrPhr_C': 0.20430107526881722, 'ra_PrNoP_C': 0.5, 'ra_PrVeP_C': 1.2666666666666666, 'ra_PrSuP_C': 19.0, 'ra_PrAjP_C': 0.0, 'ra_PrAvP_C': 6.333333333333333, 'to_AjPhr_C': 0, 'as_AjPhr_C': 0.0, 'at_AjPhr_C': 0.0, 'ra_AjNoP_C': 0.0, 'ra_AjVeP_C': 0.0, 'ra_AjSuP_C': 0.0, 'ra_AjPrP_C': 0.0, 'ra_AjAvP_C': 0.0, 'to_AvPhr_C': 3, 'as_AvPh

0    19.0
Name: content, dtype: float64

In [ ]:
df["content"][459]

'The United States dollar (sign: $; code: USD; also abbreviated US$ and referred to as the dollar, U.S. dollar, or American dollar) is the official currency of the United States and its insular territories. It is a Federal Reserve Note and consists of 100 smaller cent units.\nThe U.S. dollar is fiat money. It is the currency most used in international transactions and is the world\'s primary reserve currency.[8] Several countries use it as their official currency, and in many others it is the de facto currency.[9] Besides the United States, it is also used as the sole currency in two British Overseas Territories in the Caribbean: the British Virgin Islands and Turks and Caicos Islands. A few countries use only the U.S. dollar for paper money, while the country mints its own coins, or also accepts U.S. coins that can be used as payment in U.S. dollars, such as the Susan B. Anthony dollar.\n\n\nArticle\xa0I, Section\xa08 of the U.S. Constitution provides that the Congress has the power "

In [ ]:
get_avg_nounphrase(df['content'][459])

{'to_NoPhr_C': 2084, 'as_NoPhr_C': 9.923809523809524, 'at_NoPhr_C': 0.4639358860195904, 'ra_NoVeP_C': 1.9549718574108819, 'ra_NoSuP_C': 15.669172932330827, 'ra_NoPrP_C': 2.3521444695259595, 'ra_NoAjP_C': 24.80952380952381, 'ra_NoAvP_C': 11.707865168539326, 'to_VePhr_C': 1066, 'as_VePhr_C': 5.076190476190476, 'at_VePhr_C': 0.2373107747105966, 'ra_VeNoP_C': 0.5115163147792706, 'ra_VeSuP_C': 8.015037593984962, 'ra_VePrP_C': 1.2031602708803613, 'ra_VeAjP_C': 12.69047619047619, 'ra_VeAvP_C': 5.98876404494382, 'to_SuPhr_C': 133, 'as_SuPhr_C': 0.6333333333333333, 'at_SuPhr_C': 0.029608192341941227, 'ra_SuNoP_C': 0.06381957773512476, 'ra_SuVeP_C': 0.124765478424015, 'ra_SuPrP_C': 0.15011286681715574, 'ra_SuAjP_C': 1.5833333333333333, 'ra_SuAvP_C': 0.7471910112359551, 'to_PrPhr_C': 886, 'as_PrPhr_C': 4.219047619047619, 'at_PrPhr_C': 0.19723953695458593, 'ra_PrNoP_C': 0.42514395393474086, 'ra_PrVeP_C': 0.8311444652908068, 'ra_PrSuP_C': 6.661654135338346, 'ra_PrAjP_C': 10.547619047619047, 'ra_PrA

9.923809523809524

In [ ]:
sys.setrecursionlimit(4000)
beginner_text = df['content'][:441].progress_apply(get_avg_nounphrase)


  0%|          | 0/441 [00:00<?, ?it/s]

NameError: ignored

In [ ]:
advanced_text = df['content'][441:].progress_apply(get_avg_nounphrase)

  0%|          | 0/423 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
sys.setrecursionlimit(4000)
df['avg_nounphrases'] = df['content'].progress_apply(get_avg_nounphrase)

  0%|          | 0/864 [00:00<?, ?it/s]

In [ ]:
datasets = [df_en, df_es, df_fr, df_it]
langs = ["english", "spanish", "french", "italian"]
#[df_ca, df_en, df_es, df_eu, df_fr, df_it]

In [ ]:
sen_len = []
for k, df in enumerate(datasets):
  sen_len = []
  for i in df['content']:
    curr_sen_len = []
    for j in sent_tokenize(i, language=langs[k]):
      curr_sen_len.append(len(word_tokenize(j)))
    mean_len = np.mean(curr_sen_len)
    sen_len.append(mean_len)
  df['mean_sentence_len'] = sen_len

In [ ]:
df_es

,content,labels,difficulty,mean_sentence_len
0,tk es dominio de primer nivel para Tokelau.\n,0,beginner,9.000000
1,"Doce (lat. ""duodecim"", dos más diez) es un núm...",0,beginner,15.500000
2,Trece es un número natural entre el doce y el ...,0,beginner,13.333333
3,Quince es un número natural entre el trece y e...,0,beginner,14.333333
4,\n,0,beginner,NaN
...,...,...,...,...
829,El yo-yo es un juguete formado por un disco de...,1,advanced,24.960000
830,Zacatecas es uno de los treinta y un estados q...,1,advanced,33.504110
831,Zagreb escucha (?·i) es la capital y la ciuda...,1,advanced,24.462500
832,"Daucus carota subespecie sativus, llamada popu...",1,advanced,28.288889


In [ ]:
def walk_tree(node, depth):
    if node.n_lefts + node.n_rights > 0:
        return max(walk_tree(child, depth + 1) for child in node.children)
    else:
        return depth

def get_mean_tree_height(text):
  depths = []
  sentences = sent_tokenize(text)
  for sentence in sentences:
    doc = lang_nlp(sentence)
    depth = [walk_tree(sent.root, 0) for sent in doc.sents]
    depths.extend(depth)
  return np.mean(depths)

In [ ]:
!python -m spacy download ca_core_news_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download es_core_news_sm

2023-12-19 09:40:37.168590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 09:40:37.168648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 09:40:37.170116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 09:40:39.209512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 30.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
2023-12-19 09:41:13.384344: E external/local_xla/xla/strea

In [ ]:
lang_nlp = spacy.load("ca_core_news_sm")
df_ca['mean_tree_height'] = df_ca['content'].progress_apply(get_mean_tree_height)

  0%|          | 0/856 [00:00<?, ?it/s]

In [ ]:
lang_nlp = spacy.load('en_core_web_sm')
df_en['mean_tree_height'] = df_en['content'].progress_apply(get_mean_tree_height)

  0%|          | 0/864 [00:00<?, ?it/s]

In [ ]:
lang_nlp = spacy.load("es_core_news_sm")
df_es['mean_tree_height'] = df_es['content'].progress_apply(get_mean_tree_height)

  0%|          | 0/834 [00:00<?, ?it/s]

In [ ]:
lang_nlp = spacy.load("fr_core_news_sm")
df_fr['mean_tree_height'] = df_fr['content'].progress_apply(get_mean_tree_height)

In [ ]:
lang_nlp = spacy.load("it_core_news_sm")
df_it['mean_tree_height'] = df_it['content'].progress_apply(get_mean_tree_height)

In [10]:
root_path = "/content/drive/MyDrive/vikiwiki"

In [ ]:
df_ca = pd.read_csv(os.path.join(root_path, "df_ca.csv"))
df_ca

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,Àfrica és el tercer continent més gran del món...,0,beginner,16.666667,3.000000,4.500000
1,"L'àrab és una llengua semítica, com l'hebreu. ...",0,beginner,13.333333,3.166667,3.166667
2,Àsia és el continent més gran i poblat de la T...,0,beginner,20.000000,3.000000,1.250000
3,Ígor Stravinski (1882 - 1971) va ser un compos...,0,beginner,20.333333,4.000000,2.800000
4,Úmbria (en italià: Umbria) és una regió d'Itàl...,0,beginner,9.000000,2.666667,2.000000
...,...,...,...,...,...,...
850,"Xile, oficialment la República de Xile (Repúbl...",1,advanced,27.313492,4.573585,5.394737
851,"Yaren, abans anomenat Makwa, és un districte i...",1,advanced,19.000000,3.444444,4.000000
852,Iuri Aleksèievitx Gagarin (en rus: Ю́рий Алекс...,1,advanced,26.745098,4.564815,6.192661
853,Zàmbia és un estat de l'Àfrica Austral que lim...,1,advanced,24.548673,4.622807,5.428571


In [ ]:
df_es = pd.read_csv(os.path.join(root_path, "df_es.csv"))
df_es

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,tk es dominio de primer nivel para Tokelau.\n,0,beginner,9.000000,3.000000,2.000000
1,"Doce (lat. ""duodecim"", dos más diez) es un núm...",0,beginner,15.500000,3.000000,2.400000
2,Trece es un número natural entre el doce y el ...,0,beginner,13.333333,4.333333,2.750000
3,Quince es un número natural entre el trece y e...,0,beginner,14.333333,3.333333,2.500000
4,1955 fue un año del siglo XX de 365 días.\n,0,beginner,11.000000,2.000000,1.000000
...,...,...,...,...,...,...
827,El yo-yo es un juguete formado por un disco de...,1,advanced,24.960000,4.461538,5.909091
828,Zacatecas es uno de los treinta y un estados q...,1,advanced,33.501370,5.176781,6.524457
829,Zagreb escucha (?·i) es la capital y la ciuda...,1,advanced,24.462500,4.172840,5.325000
830,"Daucus carota subespecie sativus, llamada popu...",1,advanced,28.288889,4.604167,6.255814


In [11]:
df_en = pd.read_csv(os.path.join(root_path, "df_es.csv"))
df_en

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,tk es dominio de primer nivel para Tokelau.\n,0,beginner,9.000000,3.000000,2.000000
1,"Doce (lat. ""duodecim"", dos más diez) es un núm...",0,beginner,15.500000,3.000000,2.400000
2,Trece es un número natural entre el doce y el ...,0,beginner,13.333333,4.333333,2.750000
3,Quince es un número natural entre el trece y e...,0,beginner,14.333333,3.333333,2.500000
4,1955 fue un año del siglo XX de 365 días.\n,0,beginner,11.000000,2.000000,1.000000
...,...,...,...,...,...,...
827,El yo-yo es un juguete formado por un disco de...,1,advanced,24.960000,4.461538,5.909091
828,Zacatecas es uno de los treinta y un estados q...,1,advanced,33.501370,5.176781,6.524457
829,Zagreb escucha (?·i) es la capital y la ciuda...,1,advanced,24.462500,4.172840,5.325000
830,"Daucus carota subespecie sativus, llamada popu...",1,advanced,28.288889,4.604167,6.255814


In [ ]:
df_eu = pd.read_csv(os.path.join(root_path, "df_eu.csv"))
df_eu

,content,labels,difficulty,mean_sentence_len,avg_nounphrases,mean_tree_height
0,"Ameriketako Estatu Batuak (laburtuta, AEB) Ame...",0,beginner,12.000000,2.600000,4.333333
1,"Abaraska erleek (baita liztor batzuek ere), ez...",0,beginner,19.666667,2.500000,4.333333
2,Abeltzaintza edo abelazkuntza etxe-abereak haz...,0,beginner,16.600000,3.000000,3.416667
3,"Abeltzaintza estentsiboan, ganadua elikatzeko,...",0,beginner,16.000000,1.666667,4.500000
4,"Aberea animalia ugaztun nagusia da, piztia ez ...",0,beginner,20.333333,3.500000,5.333333
...,...,...,...,...,...,...
814,"Zientzialari, zentzu zabalean, jakintza lortze...",1,advanced,14.600000,3.272727,4.000000
815,Zigilua (edo seilua[1]) posta bidez bidali beh...,1,advanced,17.318182,3.592593,3.884615
816,"Zimbabwe[1], Afrikako hegoaldeko estatua, itsa...",1,advanced,16.968750,3.189944,3.652174
817,"Zuberoa (zubereraz, Xiberua edo Xiberoa; frant...",1,advanced,17.347222,3.320175,3.591837


In [12]:
df_fr = pd.read_csv(os.path.join(root_path, "df_fr.csv"))
df_fr

,content,labels,difficulty,mean_sentence_len,mean_tree_height,avg_nounphrases
0,Le copyright désigne le droit de l'auteur ou d...,0,beginner,17.750000,4.500000,3.200000
1,L'Édit de Milan est un ensemble de mesures pri...,0,beginner,19.000000,3.615385,3.750000
2,L'élection présidentielle française de 2002 es...,0,beginner,19.750000,4.133333,4.062500
3,Le cerveau est un organe situé dans le crâne e...,0,beginner,21.485714,3.837209,4.166667
4,"Élisabeth II, née Elizabeth Alexandra Mary le ...",0,beginner,22.111111,3.153846,3.583333
...,...,...,...,...,...,...
751,Versailles (prononcer [vɛʁ.sɑj]) est une commu...,1,advanced,30.472362,4.395538,4.484018
752,Si vous disposez d'ouvrages ou d'articles de r...,1,advanced,32.539683,4.972028,5.257576
753,"En Belgique, une ville est une commune portant...",1,advanced,25.000000,5.230769,3.666667
754,Windows Phone est un système d'exploitation mo...,1,advanced,27.508591,4.271739,4.428571


In [ ]:
df_it = pd.read_csv(os.path.join(root_path, "df_it.csv"))
df_it

,content,labels,difficulty,mean_sentence_len,avg_nounphrases,mean_tree_height
0,Água Boa è un comune di 18.994 abitanti (2007)...,0,beginner,19.000000,2.000000,4.000000
1,.bf è il dominio di primo livello nazionale (c...,0,beginner,15.000000,1.333333,3.000000
2,.br è il dominio di primo livello nazionale (c...,0,beginner,39.000000,3.333333,6.000000
3,.mx è il dominio di primo livello nazionale (c...,0,beginner,15.000000,1.333333,3.000000
4,.ru è il dominio di primo livello nazionale (c...,0,beginner,15.000000,2.000000,1.500000
...,...,...,...,...,...,...
835,"Vittorio Gassman, nato Gassmann (Struppa, 1º s...",1,advanced,33.535211,4.869565,4.505747
836,Wonkifong è un comune (in francese sous-préfec...,1,advanced,23.000000,3.000000,6.000000
837,Yosemite Sam (tradotto in alcuni cartoni come ...,1,advanced,23.315789,5.150000,4.000000
838,"Arthur Antunes Coimbra, meglio noto come Zico,...",1,advanced,28.666667,4.970588,4.684211


In [13]:
from transformers import BertTokenizer,BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup


In [14]:
class DiffDataset(Dataset):
    def __init__(self, contents, labels, features, tokenizer, max_len):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer
        self.features = features
        self.max_len = max_len

    def __len__(self):
        return len(self.contents)

    def __getitem__(self, item):
        content = str(self.contents[item])
        labels = self.labels[item]
        features = self.features[item]
        # Tokenizing the texts, while also including special tokens
        # for start and end of the text, as well as padding
        encoding = self.tokenizer.encode_plus(
          content,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt', # We return here the data as Pytorch Tensor
        )

        return {
          'content': content,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(labels, dtype=torch.long),
          'features': torch.tensor([features], dtype=torch.long)
        }


In [15]:
torch.tensor([[12, 23, 45]], dtype=torch.long).shape

torch.Size([1, 3])

In [16]:
# df_temp, df_test = train_test_split(df, test_size=0.2, random_state=42)
# df_train, df_val = train_test_split(df_temp, test_size=0.1, random_state=42)

# print("size of training set : " + str(df_train.shape[0]))
# print("size of validation set : " + str(df_val.shape[0]))
# print("size of test set : " + str(df_test.shape[0]))



In [17]:
def generate_dataloader(df, field, tokenizer, max_len, batch_size):
  ds = DiffDataset(
      contents=df.content.to_numpy(),
      labels=df.labels.to_numpy(),
      features = df[field].to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len,
    )
  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=2
    )

def generate_dataloader_1(df, tokenizer, max_len, batch_size):
  ds = DiffDataset(
      contents=df.content.to_numpy(),
      labels=df.labels.to_numpy(),
      features = df.mean_sentence_len.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len,
    )
  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=2
    )


def generate_dataloader_2(df, tokenizer, max_len, batch_size):
  ds = DiffDataset(
      contents=df.content.to_numpy(),
      labels=df.labels.to_numpy(),
      features = df.mean_tree_height.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len,
    )
  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=2
    )

def generate_dataloader_3(df, tokenizer, max_len, batch_size):
  ds = DiffDataset(
      contents=df.content.to_numpy(),
      labels=df.labels.to_numpy(),
      features = df.avg_nounphrases.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len,
    )
  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=2
    )

In [18]:
#model_name = 'bert-base-multilingual-uncased'
model_name = 'bert-base-uncased'

In [19]:
tokenizer = BertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
max_len = 256
batch_size = 8

# train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
# val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
# test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)

#data = next(iter(train_dataloader))



In [21]:
class DiffClassifier(nn.Module):

    def __init__(self, n_classes):
        super(DiffClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        # Adding drop out, keeping 90% of the last neurons of the raw BERT model
        #self.drop = nn.Dropout(p=0.1)
        # The last linear layer for multiclass classification
        self.out = nn.Linear(self.bert.config.hidden_size+1, n_classes)

    # Forward propagation function
    def forward(self, input_ids, attention_mask, features):
        model_outs = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = model_outs[0]
        pooled_output = model_outs[1]

        x = torch.cat((pooled_output, features) , dim=1)

        return self.out(x)


In [22]:
len(diff)

2

In [23]:
# Computing multiclass precision for the outputs of the model
def compute_precision(outputs, labels):
  op = outputs.cpu()
  la = labels.cpu()
  _, preds = torch.max(op, dim=1)
  # We choose 'weighted' averaging of the precision of each label because it takes into account the imbalance of labels in our tweets dataset
  # other viable averaging methods are 'micro'
  return torch.tensor(precision_score(la, preds, average='weighted',zero_division=0))

In [24]:
def train(model, train_dataloader,optimizer,scheduler, loss_fn, df_train):

        # Reset tracking variables at the beginning of each epoch
        precision, correct_predictions, batch_counts = 0, 0, 0
        losses = []

        # Put the model into the training mode
        model.train()


        # For each batch of training data...
        for d in train_dataloader:
              batch_counts +=1
              # Load batch to GPU
              input_ids = d["input_ids"].to(device)
              attention_mask = d["attention_mask"].to(device)
              labels = d["labels"].to(device)
              features = d["features"].to(device)

              outputs = model(input_ids=input_ids,attention_mask=attention_mask, features=features)

              _, preds = torch.max(outputs, dim=1)

              # Compute loss and accumulate the loss values

              loss = loss_fn(outputs, labels)

              correct_predictions += torch.sum(preds == labels)
              losses.append(loss.item())
              precision +=  compute_precision(outputs, labels)

              loss.backward()

              # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
              # Update parameters and the learning rate
              optimizer.step()
              scheduler.step()
              optimizer.zero_grad()
          # Accuracy, loss, precision
        return correct_predictions.double() / len(df_train), np.mean(losses), precision/batch_counts

In [25]:
def eval(model, valid_dataloader, loss_fn, device, n):
    model = model.eval()


    correct_predictions , precision ,batch_counts = 0,0,0
    losses = []

    with torch.no_grad():
        for d in valid_dataloader:
            batch_counts += 1

            # Preparing inputs
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            features = d["features"].to(device)

            # Running inference using the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                features=features
              )

            # Running softmax on the outputs
            _, preds = torch.max(outputs, dim=1)

            # Computing loss function
            loss = loss_fn(outputs, labels)

            # Counting the correct occurences
            correct_predictions += torch.sum(preds == labels)

            # Computing the precision (true positives/true positives + false positives)
            # for each class and label, and find their average weighted by support
            precision += compute_precision(outputs,labels)

            losses.append(loss.item())
    # Accuracy, loss, precision
    return correct_predictions.double()/n, np.mean(losses), precision/batch_counts

In [26]:
def testing(model, dataloader):
  model = model.eval()

  contents = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in dataloader:

      texts = d["content"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["labels"].to(device)
      features = d["features"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        features=features
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      contents.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(labels)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return contents, predictions, prediction_probs, real_values

In [27]:
best_model_path = "./bert_model_baseline.bin"
def save_best_model(model):
  torch.save(model.state_dict(), best_model_path)

def load_best_model(model):
  model.load_state_dict(torch.load(best_model_path))
  return model

In [28]:
class EarlyStopping:
    def __init__(self, model, patience=1):
        self.patience = patience  # number of times to allow for no improvement before stopping the execution
        self.counter = 0  # count the number of times the validation accuracy not improving
        self.min_validation_loss = np.inf
        self.model = model
        self.max_acc = 0

    # return True when validation loss is not decreased by the `min_delta` for `patience` times
    def early_stop_check(self, val_acc):
        if (val_acc > self.max_acc):
            self.max_acc = val_acc
            self.counter = 0  # reset the counter if validation loss decreased at least by min_delta
            save_best_model(self.model)
        else:
            self.counter += 1 # increase the counter if validation loss is not decreased by the min_delta
            if self.counter >= self.patience:
                return True
        return False

In [29]:
def train_loop(df: pd.DataFrame, field:str, generate_dataloader):

  b_accuracy = 0
  epochs = 10
  n_fold = 10
  random_state = 42
  folds_acc = []
  histories = []


  k_fold = StratifiedKFold(n_splits=n_fold, random_state=random_state, shuffle=True)
  y = df.drop(["content", "difficulty", field], axis=1)

  for i, (temp_index, test_index) in enumerate(k_fold.split(df, y)):
      print(" ------   Fold {}  ------- ".format(i+1), end="\n")
      df_temp = df.iloc[temp_index]
      df_train, df_val = train_test_split(df_temp, test_size=0.2, random_state=42)
      df_test = df.iloc[test_index]
      train_dataloader = generate_dataloader(df_train, tokenizer, max_len,batch_size)
      val_dataloader = generate_dataloader(df_val, tokenizer, max_len,batch_size)
      test_dataloader = generate_dataloader(df_test, tokenizer, max_len,batch_size)
      #train_dataloader = generate_dataloader(df_train, field, tokenizer, max_len,batch_size)
      #val_dataloader = generate_dataloader(df_val, field, tokenizer, max_len,batch_size)
      #test_dataloader = generate_dataloader(df_test,field, tokenizer, max_len,batch_size)


      data = next(iter(train_dataloader))
      input_ids = data['input_ids'].to(device)
      attention_mask = data['attention_mask'].to(device)
      features = data['features'].to(device)

      model = DiffClassifier(len(diff))
      # Running the classifier on GPU
      model = model.to(device)

      optimizer = AdamW(model.parameters(),
                    lr=2e-5,
                    weight_decay = 0.2,
                    correct_bias=False)
      scheduler = get_linear_schedule_with_warmup(optimizer,
                                                  num_warmup_steps=0.1,
                                                  num_training_steps=len(train_dataloader)*epochs)
      loss_fn = nn.CrossEntropyLoss().to(device)

      history = defaultdict(list)

      early_stopping = EarlyStopping(model, patience=4)

      for epoch in range(epochs):

        print(f'Epoch {epoch + 1}/{epochs}')
        print('-' * 20)

        train_acc, train_loss, train_preci = train(
              model,
              train_dataloader,
              optimizer,
              scheduler,
              loss_fn,
              df_train
            )
        print(f"Train : Loss {train_loss}, Accuracy : {train_acc*100:.2f} %, Precision : {train_preci}")

        history['train_acc'].append(train_acc)
        history['train_loss'].append(train_loss)
        history['train_precision'].append(train_preci)

        val_acc, val_loss, val_preci = eval(
              model,
              val_dataloader,
              loss_fn,
              device,
              len(df_val),
            )

        print(f'Val : Loss :{val_loss}, Accuracy : {val_acc*100:.2f} %, Precision : {val_preci}')
        print()

        history['val_acc'].append(val_acc)
        history['val_loss'].append(val_loss)
        history['val_precision'].append(val_preci)

        histories.append(history)

        if early_stopping.early_stop_check(val_acc):
          break

        # if val_acc > b_accuracy:
        #   torch.save(model.state_dict(), best_model_path)
        #   b_accuracy = val_acc

      model = load_best_model(model)
      y_contents, y_pred, y_pred_probs, y_test = testing(
            model,
            test_dataloader
          )
      acc = accuracy_score(y_test, y_pred)
      print("Test Accuracy for fold {}: {:.2f}%\n".format(i+1, acc*100))
      folds_acc.append(acc)
  return folds_acc, histories


Basque Language


In [ ]:
df = df_eu

In [ ]:
mean_sen_folds_acc, mean_sen_histories =  train_loop(df.drop(["mean_tree_height", "avg_nounphrases"], axis=1), "mean_sentence_len", generate_dataloader_1)

In [ ]:
print("The accuracy score for mean sentence length for Basque Language is: {:.3f}%".format(sum(mean_sen_folds_acc) * 100 / len(mean_sen_folds_acc)))


The accuracy score for mean sentence length for Basque Language is: 65.673%


In [ ]:
mean_tree_height_folds_acc, mean_tree_height_histories = train_loop(df.drop(["mean_sentence_len", "avg_nounphrases"], axis=1), "mean_tree_height", generate_dataloader_2)

In [ ]:
print("The accuracy score for mean_tree_height feature for Basque Language is: {:.3f}%".format(sum(mean_tree_height_folds_acc) * 100 / len(mean_tree_height_folds_acc)))

The accuracy score for mean_tree_height feature for Basque Language is: 70.795%


In [ ]:
avg_noun_folds_acc, avg_noun_histories =  train_loop(df.drop(["mean_sentence_len", "mean_tree_height"], axis=1), "avg_nounphrases", generate_dataloader_3)

In [ ]:
print("The accuracy score for avg_nounphrases feature for Basque Language is: {:.3f}%".format(sum(avg_noun_folds_acc) * 100 / len(avg_noun_folds_acc)))

The accuracy score for avg_nounphrases feature for Basque Language is: 59.453%


Catalan Language

In [ ]:
df = df_ca

In [ ]:
mean_sen_folds_acc, mean_sen_histories =  train_loop(df.drop(["mean_tree_height", "avg_nounphrases"], axis=1), "mean_sentence_len", generate_dataloader_1)

In [ ]:
print("The accuracy score for mean sentence length for Catalan Language is: {:.3f}%".format(sum(mean_sen_folds_acc) * 100 / len(mean_sen_folds_acc)))


The accuracy score for mean sentence length for Catalan Language is: 96.140%


In [ ]:
mean_tree_height_folds_acc, mean_tree_height_histories = train_loop(df.drop(["mean_sentence_len", "avg_nounphrases"], axis=1), "mean_tree_height", generate_dataloader_2)

In [ ]:
print("The accuracy score for mean_tree_height feature for Catalan Language is: {:.3f}%".format(sum(mean_tree_height_folds_acc) * 100 / len(mean_tree_height_folds_acc)))

The accuracy score for mean_tree_height feature for Catalan Language is: 94.850%


In [ ]:
avg_noun_folds_acc, avg_noun_histories =  train_loop(df.drop(["mean_sentence_len", "mean_tree_height"], axis=1), "avg_nounphrases", generate_dataloader_3)

In [ ]:
print("The accuracy score for avg_nounphrases feature for Catalan Language is: {:.3f}%".format(sum(avg_noun_folds_acc) * 100 / len(avg_noun_folds_acc)))

The accuracy score for avg_nounphrases feature for Catalan Language is: 96.138%


English Language

In [30]:
df = df_en

In [ ]:
mean_sen_folds_acc, mean_sen_histories =  train_loop(df.drop(["mean_tree_height"], axis=1), "mean_sentence_len", generate_dataloader_1)

In [ ]:
print("The accuracy score for mean sentence length for English Language is: {:.3f}%".format(sum(mean_sen_folds_acc) * 100 / len(mean_sen_folds_acc)))


In [ ]:
mean_tree_height_folds_acc, mean_tree_height_histories = train_loop(df.drop(["mean_sentence_len"], axis=1), "mean_tree_height", generate_dataloader_2)

In [ ]:
print("The accuracy score for mean_tree_height feature for English Language is: {:.3f}%".format(sum(mean_tree_height_folds_acc) * 100 / len(mean_tree_height_folds_acc)))

The accuracy score for mean_tree_height_feature is: 95.491%


In [31]:
avg_noun_folds_acc, avg_noun_histories =  train_loop(df.drop(["mean_sentence_len", "mean_tree_height"], axis=1), "avg_nounphrases", generate_dataloader_3)

 ------   Fold 1  ------- 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/10
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Train : Loss 0.0685181978577748, Accuracy : 98.83 %, Precision : 0.9914126984126983


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.8975865945134214, Accuracy : 85.33 %, Precision : 0.8846491228070174



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Test Accuracy for fold 10: 90.36%



In [32]:
print("The accuracy score for avg_nounphrases feature for English Language is: {:.3f}%".format(sum(avg_noun_folds_acc) * 100 / len(avg_noun_folds_acc)))


The accuracy score for avg_nounphrases feature for English Language is: 88.699%


French Language

In [33]:
df = df_fr

In [ ]:
mean_sen_folds_acc, mean_sen_histories =  train_loop(df.drop(["mean_tree_height", "avg_nounphrases"], axis=1), "mean_sentence_len", generate_dataloader_1)

In [ ]:
print("The accuracy score for mean sentence length for French Language is: {:.3f}%".format(sum(mean_sen_folds_acc) * 100 / len(mean_sen_folds_acc)))

In [ ]:
mean_tree_height_folds_acc, mean_tree_height_histories = train_loop(df.drop(["mean_sentence_len", "avg_nounphrases"], axis=1), "mean_tree_height", generate_dataloader_2)

In [ ]:
print("The accuracy score for mean_tree_height_feature for French Language is: {:.3f}%".format(sum(mean_tree_height_folds_acc) * 100 / len(mean_tree_height_folds_acc)))

In [34]:
avg_noun_folds_acc, avg_noun_histories =  train_loop(df.drop(["mean_sentence_len", "mean_tree_height"], axis=1), "avg_nounphrases", generate_dataloader_3)

 ------   Fold 1  ------- 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/10
--------------------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Train : Loss 0.7207077233230367, Accuracy : 50.92 %, Precision : 0.35474877450980397


...

Train : Loss 0.019858975413679966, Accuracy : 99.63 %, Precision : 0.996953781512605


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val : Loss :0.44822004824689227, Accuracy : 92.70 %, Precision : 0.9458333333333333



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Test Accuracy for fold 10: 88.00%



In [35]:
print("The accuracy score for avg_nounphrases feature for French Language is: {:.3f}%".format(sum(avg_noun_folds_acc) * 100 / len(avg_noun_folds_acc)))

The accuracy score for avg_nounphrases feature for French Language is: 84.412%


Italian Language

In [ ]:
df = df_it

In [ ]:
mean_sen_folds_acc, mean_sen_histories =  train_loop(df.drop(["mean_tree_height", "avg_nounphrases"], axis=1), "mean_sentence_len", generate_dataloader_1)

In [ ]:
print("The accuracy score for mean sentence length for Italian Language is: {:.3f}%".format(sum(mean_sen_folds_acc) * 100 / len(mean_sen_folds_acc)))

The accuracy score for mean sentence length for Italian Language is: 91.190%


In [ ]:
mean_tree_height_folds_acc, mean_tree_height_histories = train_loop(df.drop(["mean_sentence_len", "avg_nounphrases"], axis=1), "mean_tree_height", generate_dataloader_2)

In [ ]:
print("The accuracy score for mean_tree_height_feature for Italian Language is: {:.3f}%".format(sum(mean_tree_height_folds_acc) * 100 / len(mean_tree_height_folds_acc)))

The accuracy score for mean_tree_height_feature for Italian Language is: 90.833%


In [ ]:
avg_noun_folds_acc, avg_noun_histories =  train_loop(df.drop(["mean_sentence_len", "mean_tree_height"], axis=1), "avg_nounphrases", generate_dataloader_3)

In [ ]:
print("The accuracy score for avg_nounphrases feature for Italian Language is: {:.3f}%".format(sum(avg_noun_folds_acc) * 100 / len(avg_noun_folds_acc)))

The accuracy score for avg_nounphrases feature for Italian Language is: 91.310%


Spanish Language

In [ ]:
df = df_es

In [ ]:
mean_sen_folds_acc, mean_sen_histories =  train_loop(df.drop(["mean_tree_height", "avg_nounphrases"], axis=1), "mean_sentence_len", generate_dataloader_1)

In [ ]:
print("The accuracy score for mean sentence length for Spanish Language is: {:.3f}%".format(sum(mean_sen_folds_acc) * 100 / len(mean_sen_folds_acc)))

The accuracy score for mean sentence length for Spanish Language is: 86.427%


In [ ]:
mean_tree_height_folds_acc, mean_tree_height_histories = train_loop(df.drop(["mean_sentence_len", "avg_nounphrases"], axis=1), "mean_tree_height", generate_dataloader_2)

In [ ]:
print("The accuracy score for mean_tree_height_feature for Spanish Language is: {:.3f}%".format(sum(mean_tree_height_folds_acc) * 100 / len(mean_tree_height_folds_acc)))

The accuracy score for mean_tree_height_feature for Spanish Language is: 86.905%


In [ ]:
avg_noun_folds_acc, avg_noun_histories =  train_loop(df.drop(["mean_sentence_len", "mean_tree_height"], axis=1), "avg_nounphrases", generate_dataloader_3)

In [ ]:
print("The accuracy score for avg_nounphrases feature for Spanish Language is: {:.3f}%".format(sum(avg_noun_folds_acc) * 100 / len(avg_noun_folds_acc)))

The accuracy score for avg_nounphrases feature for Spanish Language is: 87.250%
